In [1]:
import requests
from bs4 import BeautifulSoup

import os
import re
import pickle

from tqdm import tqdm, trange

In [2]:
GENIUS_API_TOKEN='0DRkNDsVx6eWmcAXzWNLct1DPEQ24l79DHvie_0YeGtXfih-pcl6BNO0uBVCIxQm'

In [3]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    conn_timeout = 1
    read_timeout = 10
    timeouts = (conn_timeout, read_timeout)
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers, timeout=timeouts)
    return response

# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    zero_iter = 0
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
                
        if not len(song_info):
            zero_iter += 1
        if zero_iter >= 3:
            break
#         print(f'song_info on page {page}: {len(song_info)}')

        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                # print(song['result'])
                title = song['result']['title']
                url = song['result']['url']
                songs.append((title, url))
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    # print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
# request_song_url('j balvin', 2)

In [11]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    conn_timeout = 1
    read_timeout = 10
    timeouts = (conn_timeout, read_timeout)
    page = requests.get(url, timeout=timeouts)
    html = BeautifulSoup(page.text, 'html.parser')
    try:
        lyrics = html.find('div', class_='lyrics').get_text()
        #remove identifiers like chorus, verse, etc
        lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
        #remove empty lines
        lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])      
    except AttributeError:
        lyrics = ''   
    return lyrics

# # DEMO
# scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics')

In [5]:
def get_lyrics(artist_name, song_count):
    output = []
    res = request_song_url(artist_name, song_count)
    if not res: return []
    
    titles, urls = list(zip(*res))
    for i, url in enumerate(tqdm(urls)):
        lyrics = scrape_song_lyrics(url)
        output.append((titles[i], lyrics))
    
    if not output:
        print(f'no lyrics for {artist_name}')
    return output

In [12]:
new_a = []
with open('new_artists.txt', 'r+') as fp:
    for line in fp:
        new_a.append(line.strip('\n'))


In [13]:
new_a[:5]

['? & The Mysterions',
 'Aaron Neville',
 'Archie Bell & The Drells',
 'Arthur Alexander',
 'Arthur Conley']

In [16]:

index = 128
for artist in new_a[index:]:
    index += 1
    print(index, artist)
    if artist in res:
        continue
    res[artist] = get_lyrics(artist, 30)

129 The Kingsmen
100%|██████████| 30/30 [00:17<00:00,  1.71it/s]
130 The Kinks
100%|██████████| 30/30 [00:15<00:00,  1.89it/s]
131 The Lovin' Spoonful
132 The MC5
133 The Mamas & The Papas
100%|██████████| 30/30 [00:23<00:00,  1.28it/s]
134 The Meters
100%|██████████| 30/30 [00:38<00:00,  1.29s/it]
135 The Monkees
100%|██████████| 30/30 [00:31<00:00,  1.04s/it]
136 The Moody Blues
100%|██████████| 30/30 [00:27<00:00,  1.10it/s]
137 The Olympics
138 The Orlons
100%|██████████| 11/11 [00:13<00:00,  1.18s/it]
139 The Pretty Things
100%|██████████| 30/30 [00:43<00:00,  1.44s/it]
140 The Righteous Brothers
100%|██████████| 30/30 [00:32<00:00,  1.08s/it]
141 The Shangri-Las
100%|██████████| 30/30 [00:31<00:00,  1.05s/it]
142 The Sonics
100%|██████████| 30/30 [00:45<00:00,  1.52s/it]
143 The Spencer Davis Group
100%|██████████| 30/30 [00:37<00:00,  1.24s/it]
144 The Staple Singers
100%|██████████| 30/30 [00:32<00:00,  1.07s/it]
145 The Stooges
100%|██████████| 30/30 [00:44<00:00,  1.49s/it]
1

ReadTimeout: HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=10)

In [15]:
len(res)

128